In [1]:
import os
import sqlite3
import pandas as pd

# SQL

**SQL** (**S**tructured **Q**uery **L**anguage) is a language for interacting with [relational databases](https://en.wikipedia.org/wiki/Relational_database). As with so many things, there is a Python library that allows us to execute SQL commands from Python. In fact there are a few such libraries, but we will focus here on one of the simplest, `sqlite3`, which interfaces with a popular variant of SQL called 'SQLite'.

We will demonstrate just a few of the basics of SQL. To learn more, see the following links:

* SQLite main [documentation page](https://www.sqlite.org/index.html)
* a great SQL [tutorial](https://www.w3schools.com/sql/default.asp)

## Relational databases

We are familiar with the idea of storing data in tables, whose rows represent units of observation (such as a person, a point in time, a measurement, a company, etc.) and whose columns represent variables, i.e. pieces of information for each of the units of observation (such as age, Reaction Time, symptom severity, turnover, etc.)

One of the most important features of a relational database is that it can (and almost almost always does) consist of more than one such table, where the tables are in some way related to one another. This model for representing data has some advantages over single tables. To get an idea of why, let's look at an example of a single table of data stored in a csv file.

The file stores the names of songs in a small database of music. There are columns for the artist and album, along with some other information about each.

In [2]:
table = pd.read_csv('music.csv')
print(table.head())

             Artist Country                    Album  Year  \
0  Blue Öyster Cult      US  The Revolution By Night  1987   
1  Blue Öyster Cult      US  The Revolution By Night  1987   
2  Blue Öyster Cult      US  The Revolution By Night  1987   
3  Blue Öyster Cult      US  The Revolution By Night  1987   
4  Blue Öyster Cult      US  The Revolution By Night  1987   

                   Song  Number  
0          Take Me Away       1  
1          Eyes On Fire       2  
2        Shooting Shark       3  
3                 Veins       4  
4  Shadow of California       5  


One thing that is immediately apparent is that the file duplicates a lot of information. Each row is a single song, so song is our 'lowest level' unit of observation. But many of the songs come from the same album, and some of the albums are by the same artist. To represent this fact in a single table, we have to repeat artist and album information on every row, even if that same information is already present in the table elsewhere. Note also that we have no easy way of asserting that these repetitions of artist and album names really do refer to the same entity, and not for example to two different albums that happen to have the same name.

This is not necessarily a bad way of organizing data. If our set of data is small and we do not care too much about being able to store it efficiently and access it quickly, then a single table is often fine. However, for very big databases that we need to access really fast, for example on a busy web server, this model is inefficient. It is also prone to error should we mistakenly insert the same data twice or forget to take repetitions into account when counting artists or albums.

This is where a relational database comes in. It allows us to store data with multiple units of observation, without repeating information about each unit. In the example above, our multiple units of observation are songs, albums, and artists, so we could have a table for each. This sort of a database conforms to one of the [first basic principles of efficient data storage](https://en.wikipedia.org/wiki/First_normal_form): don't repeat information about the same unit of observation. As long as we take care to link the tables appropriately, the full set of information relevant to one or more observations can be joined together if needed.

## Reading from a database

We will now take a look at an example SQLite database that stores the same information as the csv table above. To get an easier, interactive overview of the contents of the database file, you can first view it using a graphical browser for SQLite databases. [Here](https://sqlitebrowser.org/) is a good one. (Be sure to close the database in the browser before trying to interact with it in Python, or you may not be able to access the file, or may access a temporary copy of it.)

The first step in Python is to open the database file. Then we insert a 'cursor' into the file. This cursor acts as our agent in the file, and we can instruct it to read (or write) data.

In [3]:
conn = sqlite3.connect('music.db')
cursor = conn.cursor()

### SELECT

The database contains three tables, for artists, albums, and songs. The SQL `SELECT` and `FROM` keywords select columns of data and specify from which table. The wildcard `*` means 'all', as it does in various other programming environments.

(Note that SQL keywords do not have to be written in uppercase, but it is a common convention to do so, in order to distinguish them clearly from the names of tables and columns.)

In [4]:
statement = "SELECT * FROM artists"
cursor.execute(statement)

Once we have selected data, the Python cursor object becomes iterable, and we can use it to loop through the selected rows of data.

In [5]:
for row in cursor:
    print(row)

(1, 'Blue Öyster Cult', 'US')
(2, 'Geier Sturzflug', 'DE')
(3, 'Steeleye Span', 'GB')


Alternatively, if we know that we will want all the rows, we can get them in a single list.

In [6]:
cursor.execute(statement)
print(cursor.fetchall())

[(1, 'Blue Öyster Cult', 'US'), (2, 'Geier Sturzflug', 'DE'), (3, 'Steeleye Span', 'GB')]


Or if we want just one row, there is a method for that too.

In [7]:
cursor.execute(statement)
print(cursor.fetchone())

(1, 'Blue Öyster Cult', 'US')


The cursor fetches rows of data as tuples, without information about column names. Though this leaves the output somewhat ambiguous, it is more efficient than storing column information along with every row of data. If we are interacting with a database that we have created ourselves (and if we have written our program carefully), we will know what the columns represent, and we will have built this information into our program.

If we need to check overall information about a database, the SQLite `PRAGMA` commands can retrieve it.

In [8]:
statement = "PRAGMA table_info('artists')"
cursor.execute(statement)

for row in cursor:
    print(row)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'artist', 'TEXT', 0, None, 0)
(2, 'country', 'TEXT', 0, None, 0)


We can see that along with the columns for artist name and country that were present in the csv file, there is an integer id for each artist. This is a number that the SQL database has assigned automatically to each row of data entered into the table, ensuring that each can be referenced by a unique number. In SQL terminology, a column like this is known as a `PRIMARY KEY`.

### JOIN

Now let's look at the table of albums. First the table information:

In [9]:
statement = "PRAGMA table_info('albums')"
cursor.execute(statement)

for row in cursor:
    print(row)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'artist_id', 'INTEGER', 0, None, 0)
(2, 'album', 'TEXT', 0, None, 0)
(3, 'year', 'INTEGER', 0, None, 0)


Then the data themselves:

In [10]:
statement = "SELECT * FROM albums"
cursor.execute(statement)

for row in cursor:
    print(row)

(1, 1, 'The Revolution By Night', 1987)
(2, 1, 'Club Ninja', 2009)
(3, 2, 'Heiße Zeiten', 1983)
(4, 2, 'Lust am Leben', 2001)
(5, 3, 'Below the Salt', 1972)
(6, 3, 'Hark! The Village Wait', 2006)


Here we see that in addition to its own id column, the albums table has a column referencing the id number of the artist for the album. This column provides a link back to the table of artists. Instead of repeating all of the information about artists, the albums table links to it via a single integer. This is more efficient than storing this information as repeated text, since computers can store integers more compactly and look them up quicker.

A column that references an id number in a different table is termed a `FOREIGN KEY`. These foreign keys can be used to join up the information from two or more tables if we need to reconstruct the longer format. The SQL `JOIN` keyword joins one table onto another. It is usually used in combination with `SELECT`. We specify which column in one table references whcih column in the other using the `ON` keyword and `=` to link the two columns.

In [11]:
statement = "SELECT * FROM albums JOIN artists ON albums.artist_id = artists.id"
cursor.execute(statement)

for row in cursor:
    print(row)

(1, 1, 'The Revolution By Night', 1987, 1, 'Blue Öyster Cult', 'US')
(2, 1, 'Club Ninja', 2009, 1, 'Blue Öyster Cult', 'US')
(3, 2, 'Heiße Zeiten', 1983, 2, 'Geier Sturzflug', 'DE')
(4, 2, 'Lust am Leben', 2001, 2, 'Geier Sturzflug', 'DE')
(5, 3, 'Below the Salt', 1972, 3, 'Steeleye Span', 'GB')
(6, 3, 'Hark! The Village Wait', 2006, 3, 'Steeleye Span', 'GB')


We have seen that after selecting, the cursor fetches rows of data as tuples, without information about column names. The `*` wildcard can (and in most cases should) be replaced with the names of the columns we want to fetch. Fetching only the columns we need tends to make working with the data faster, and it will also ensure that we are clear about what information we are working with.

In [12]:
statement = "SELECT artist, album FROM albums JOIN artists ON albums.artist_id = artists.id"
cursor.execute(statement)

for row in cursor:
    print(row)

('Blue Öyster Cult', 'Club Ninja')
('Blue Öyster Cult', 'The Revolution By Night')
('Geier Sturzflug', 'Heiße Zeiten')
('Geier Sturzflug', 'Lust am Leben')
('Steeleye Span', 'Below the Salt')
('Steeleye Span', 'Hark! The Village Wait')


Multiple `JOIN`s are allowed, and will often be necessary for more complex data sets. Once our SQL statements become long, we can make them clearer by separating the string containing the statement over several lines, using the `\` character.

In [13]:
statement = "SELECT artist, album, song FROM songs \
JOIN albums ON songs.album_id = albums.id \
JOIN artists ON songs.artist_id = artists.id"

cursor.execute(statement)

for row in cursor:
    print(row)

('Blue Öyster Cult', 'The Revolution By Night', 'Take Me Away')
('Blue Öyster Cult', 'The Revolution By Night', 'Eyes On Fire')
('Blue Öyster Cult', 'The Revolution By Night', 'Shooting Shark')
('Blue Öyster Cult', 'The Revolution By Night', 'Veins')
('Blue Öyster Cult', 'The Revolution By Night', 'Shadow of California')
('Blue Öyster Cult', 'The Revolution By Night', 'Feel the Thunder')
('Blue Öyster Cult', 'The Revolution By Night', 'Let Go')
('Blue Öyster Cult', 'The Revolution By Night', 'Dragon Lady')
('Blue Öyster Cult', 'The Revolution By Night', 'Light Years of Love')
('Blue Öyster Cult', 'Club Ninja', 'White Flags')
('Blue Öyster Cult', 'Club Ninja', 'Dancin’ in the Ruins')
('Blue Öyster Cult', 'Club Ninja', 'Make Rock Not War')
('Blue Öyster Cult', 'Club Ninja', 'Perfect Water')
('Blue Öyster Cult', 'Club Ninja', 'Spy in the House of the Night')
('Blue Öyster Cult', 'Club Ninja', 'Beat ’Em Up')
('Blue Öyster Cult', 'Club Ninja', 'When the War Comes')
('Blue Öyster Cult', 'Clu

### WHERE

The final SQL keyword necessary for basic database queries is `WHERE`. This retrieves a subset of the rows based on a condition. Here as an example we select only the first song from each album.

(Note that SQL's equality operator is the single `=` not the double `==` that is used in Python.)

In [14]:
statement = "SELECT * FROM songs WHERE number = 1"

cursor.execute(statement)

for row in cursor:
    print(row)

(1, 1, 1, 'Take Me Away', 1)
(10, 1, 2, 'White Flags', 1)
(19, 2, 3, 'Besuchen Sie Europa (solange es noch steht)', 1)
(30, 2, 4, 'Pure Lust am Leben', 1)
(48, 3, 5, 'Spotted Cow', 1)
(57, 3, 6, 'A Calling-On Song', 1)


`WHERE` can of course be combined with `JOIN`.

In [15]:
statement = "SELECT artist, album, song, number FROM songs \
JOIN albums ON songs.album_id = albums.id \
JOIN artists ON songs.artist_id = artists.id \
WHERE number = 1"

cursor.execute(statement)

for row in cursor:
    print(row)

('Blue Öyster Cult', 'The Revolution By Night', 'Take Me Away', 1)
('Blue Öyster Cult', 'Club Ninja', 'White Flags', 1)
('Geier Sturzflug', 'Heiße Zeiten', 'Besuchen Sie Europa (solange es noch steht)', 1)
('Geier Sturzflug', 'Lust am Leben', 'Pure Lust am Leben', 1)
('Steeleye Span', 'Below the Salt', 'Spotted Cow', 1)
('Steeleye Span', 'Hark! The Village Wait', 'A Calling-On Song', 1)


### pandas and SQL

The pandas library provides a function for executing an SQL statement and reading the selected data into a pandas dataframe. This can be convenient if we want just one subset of data, and then want to apply plotting or statistical summaries.

(Note that the second argument to the pandas function is the **connection** object that we got when opened the database file, not the **cursor** object.)

In [16]:
statement = "SELECT artist, year FROM albums \
JOIN artists ON albums.artist_id = artists.id"

df = pd.read_sql_query(statement, conn)

print(df)

             artist  year
0  Blue Öyster Cult  1987
1  Blue Öyster Cult  2009
2   Geier Sturzflug  1983
3   Geier Sturzflug  2001
4     Steeleye Span  1972
5     Steeleye Span  2006


When we have finished working with a database, it is good practice to close the connection, so that access is freed for other applications.

In [17]:
conn.close()

## Writing to a database

We can also use SQLite to create a new database or write to an existing database. Let's first create a new database. The functions for this are the same as for connecting to an existing database.

(For convenience, here we also delete the database file if it has already been created the last time this example was run.)

In [18]:
try:
    os.remove('example.db')
except OSError:
    pass

conn = sqlite3.connect('example.db')
cursor = conn.cursor()

### CREATE TABLE

Usually the first thing we will want to do in a new database is to create some tables. We assign columns to each table we create. In doing so, there are a few useful keywords that we can use to impose some 'constraints' on the table.

* specify the [data type](https://www.w3schools.com/sql/sql_datatypes.asp) to be stored in a column
* specify which column constitutes the `PRIMARY KEY` for the table
* assert that the values in a column must be `UNIQUE` (most commonly we will want to use this for names or other identifying information that no two observations should share)

Here we will use the same example as above, to illustrate how the database was created (though we won't go all the way to adding every table and every row of data).

In [19]:
statement = "CREATE TABLE artists \
(id INTEGER PRIMARY KEY, \
artist TEXT UNIQUE, \
country TEXT)"

cursor.execute(statement)

Constraints can also be added after the column names if the constraint is more complex than a single keyword:

* `FOREIGN KEY` along with `REFERENCES` specify that a column constitutes a foreign key and which column of which other table it references
* `UNIQUE` can also apply to multiple columns, specifying that *combinations* of values must be unique

For example, we use these statements here to specify the artist key for the albums table, and to assert that no artist can have two albums with the same name (although two same-named albums by different artists would be ok).

In [20]:
statement = "CREATE TABLE albums \
(id INTEGER PRIMARY KEY, \
artist_id INTEGER, \
album TEXT, \
year INTEGER, \
FOREIGN KEY(artist_id) REFERENCES artists(id), \
UNIQUE(artist_id, album))"

cursor.execute(statement)

### INSERT

Next we will usually want to insert data into the tables. We specify which table we want to insert the data into, along with which columns, and a tuple of values to insert. It is not very convenient to have to write the values literally into the string SQL statement, since these will usually come from variables in our Python program. The cursor's `execute()` method also allows for a second argument giving a tuples of values. These will be inserted into the SQL statement where `?` characters appear.

In [21]:
statement = "INSERT INTO artists (artist, country) VALUES(?, ?)"

artist = 'Geier Sturzflug'
country = 'DE'

cursor.execute(statement, (artist, country))

Let's check that it has worked:

In [22]:
statement = "SELECT * FROM artists"
cursor.execute(statement)

for row in cursor:
    print(row)

(1, 'Geier Sturzflug', 'DE')


When we created the artists table, we asserted that the 'name' column must contain unique values (i.e. cannot contain any repeated values). We have not yet seen this constraint in action. Behind the scenes, our database has noted this constraint, and will raise an exception if we violate the constraint by inserting the same name again. Let's see what happens if we repeat inserting the same artist as above.

(To view the error concisely and without interrupting the remainder of the example, we catch the exception and just print it out.)

In [23]:
statement = "INSERT INTO artists (artist, country) VALUES(?, ?)"

artist = 'Geier Sturzflug'
country = 'DE'

try:
    cursor.execute(statement, (artist, country))
except sqlite3.IntegrityError as err:
    print(err)

UNIQUE constraint failed: artists.artist


If we are inserting values that might violate a constraint that we have set, but would prefer just to ignore such cases rather than have them interrupt our program, then we can use `INSERT OR IGNORE` in SQL.

In [24]:
statement = "INSERT OR IGNORE INTO artists (artist, country) VALUES(?, ?)"

artist = 'Geier Sturzflug'
country = 'DE'

cursor.execute(statement, (artist, country)) # no error this time

When writing to a database, SQL does not always write the results of every statement to disk when we execute it. Instead, the results of executing statements may be held in memory, to be written all in one go later. This usually makes for a faster program, since writing to disk is time-consuming. It also sometimes allows for the structure of the database file to be optimized based on all the data it should contain so far.

If we reach a point in our program where we would like to finish off a batch of writing and ensure that everything is really finalized and written to the database file, we can 'commit' the current changes. In `sqlite3` this is done via a method of the connection object (Note: not the cursor object).

In [25]:
conn.commit()

## SQL injection

Why does `sqlite3` perform string insertion its own way, using `?`; why can't we just build SQL statements using the normal Python string formatting?

We can build SQL statements with string formatting if we want to, since the statements are just strings like any other. In most cases this is totally fine, and in some cases it may even be necessary. However, if we are writing a program to process database queries that may be sent by inept or even malicious users, there is a certain risk involved in building SQL statements as strings formatted together with user input. If we allow any input to be inserted into an SQL statement, there is nothing to stop users from inserting further SQL statements instead of just data. This is an attack technique known as [SQL injection](https://en.wikipedia.org/wiki/SQL_injection).

Here is an example of injection (inspired by [this](https://xkcd.com/327/) famous illustration):

In [26]:
# imagining the following variables originated as user input:
artist = 'Rick Astley'
country = "GB'); DROP TABLE artists; --"

statement = "INSERT INTO artists (artist, country) VALUES('{}', '{}')".format(artist, country)
print(statement)

INSERT INTO artists (artist, country) VALUES('Rick Astley', 'GB'); DROP TABLE artists; --')


The `;` character in SQL indicates the end of a statement, so by inserting this character, an attacker can end our intended statement and then add on a second one of their own. The `--` characters in SQL indicate a comment, so by placing these at the end of their injected string, the attacker also prevents any remaining parts of our statement from being interpreted.

In fact, the `sqlite3` library already implements some basic protections against injection, even if we inadvertently build destructive SQL statements. The cursor's `execute()` method will only execute one statement at a time, so an injection using `;` to separate multiple statements will fail. Only if we use the `executescript()` method, which does allow multiple statements, will the example malicious statement above be effective.

In [27]:
cursor.executescript(statement)

cursor.execute("PRAGMA table_info('artists')")
print(cursor.fetchall()) # the table has gone

[]


We are done with our examples, so let's close the connection to the database file.

In [28]:
conn.close()